News Recommender

In [1]:
# Upload packages
import requests
import pandas as pd
import os

In [2]:
# Read JSON objects
key = "3026e0d9c0c94df2ac184e6b103d28c7"
usnewsapi = requests.get("https://newsapi.org/v2/top-headlines?country=gb&apiKey=3026e0d9c0c94df2ac184e6b103d28c7").json()


In [3]:
# Convert to DataFrame
usnews = pd.DataFrame(usnewsapi['articles'])

In [4]:
usnews.head(3)

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'The Guardian'}",Simon Parkin,Alex Horne on the secret to his cult show Task...,How does the comedian and creative genius come...,https://www.theguardian.com/lifeandstyle/2022/...,https://i.guim.co.uk/img/media/bbfc67819081146...,2022-12-24T12:00:00Z,"Alex Horne is sweltering. Taskmaster, the Brit..."
1,"{'id': 'financial-times', 'name': 'Financial T...","Steven Bernard, Emiliya Mychasuk",Weather warnings in US as Arctic freeze grips ...,"News, analysis and comment from the Financial ...",https://www.ft.com/content/0da5d62c-4abb-4f58-...,None,2022-12-24T10:46:32Z,What is included in my trial?\r\nDuring your t...
2,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Man arrested after Essex pub assault - BBC,A man dies after reports of an assault at the ...,https://www.bbc.co.uk/news/uk-england-essex-64...,https://ichef.bbci.co.uk/news/1024/branded_new...,2022-12-24T10:34:43Z,A man has been arrested on suspicion of murder...


In [5]:
usnews.shape

(20, 8)

In [11]:
usnews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source       20 non-null     object
 1   author       19 non-null     object
 2   title        20 non-null     object
 3   description  19 non-null     object
 4   url          20 non-null     object
 5   urlToImage   17 non-null     object
 6   publishedAt  20 non-null     object
 7   content      20 non-null     object
dtypes: object(8)
memory usage: 1.4+ KB


Content-Based Recommender

In [6]:
# Print content description of the first 5 sports news
usnews['content'].head()

0    Alex Horne is sweltering. Taskmaster, the Brit...
1    What is included in my trial?\r\nDuring your t...
2    A man has been arrested on suspicion of murder...
3    Convicted killer Charles Sobhraj has arrived i...
4    Building cars is hard, so when Ben Hedley star...
Name: content, dtype: object

In [16]:
# Import TfidfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

# Conver to list
feature = usnews['title'].tolist()

# Define a TF-IDF Vectorizer object. Remove all english stop words
tfidf = TfidfVectorizer(input=feature, stop_words='english')

# Construct the required TF-IDF matrix by fitting and transform
tfidf_matrix = tfidf.fit_transform(feature)

# Find the similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(tfidf_matrix)

In [18]:
# Construct a reverse map of indices and article titles
indices = pd.Series(usnews.index, index=usnews['title']).drop_duplicates()
print(indices[:10])

title
Alex Horne on the secret to his cult show Taskmaster: ‘It must be offbeat but not wacky, off-kilter but not bonkers’ - The Guardian    0
Weather warnings in US as Arctic freeze grips northern hemisphere - Financial Times                                                    1
Man arrested after Essex pub assault - BBC                                                                                             2
'The Serpent' killer Charles Sobhraj arrives back in France after nearly two decades - Sky News                                        3
‘Brilliant fun’: UK automaker shrinks classic cars for big spenders - The Guardian                                                     4
Live M25, M1, A1 traffic on Herts motorways as millions set to travel on Christmas Eve - Herts Live                                    5
Fire in Russian care home kills 22 people - The Guardian                                                                               6
Jonnie Irwin had 'no symptoms befor

In [21]:
# Create function that takes in article title as input and output similar articles
def news_recommendation(title, similarity=similarity):
    # Get the index of the article that matches the title
    index = indices[title]

    # Get the pairwise similarity scores of all articles
    similarity_scores = list(enumerate(similarity[index]))

    # Sort the articles based on the similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 10 most similar articles
    similarity_scores = similarity_scores[0:10]

    # Get the article indices
    article_indices = [i[0] for i in similarity_scores]

    # Return the top 10 most similar articls
    return usnews['title'].iloc[article_indices]

print(news_recommendation("Man arrested after Essex pub assault - BBC"))

2            Man arrested after Essex pub assault - BBC
9     White Christmas forecast for Scotland as weath...
8     Woman, 20, killed in block of flats as teen ar...
12    Transfer news LIVE: Man Utd battle Liverpool o...
18    Erik ten Hag responds to Cody Gakpo comment an...
0     Alex Horne on the secret to his cult show Task...
1     Weather warnings in US as Arctic freeze grips ...
3     'The Serpent' killer Charles Sobhraj arrives b...
4     ‘Brilliant fun’: UK automaker shrinks classic ...
5     Live M25, M1, A1 traffic on Herts motorways as...
Name: title, dtype: object
